# Rule Splitting & ElasticSearch Uploads
## LMI Capstone Team
## Summer Chambers | Steve Morris | Kaleb Shikur

### Setup ES

In [2]:
'''
Rule Splitting, Uploading to ES
'''
from elasticsearch import Elasticsearch, RequestsHttpConnection
# from requests_aws4auth import AWS4Auth
import numpy as np
import requests #gets urls
# from aws_requests_auth.boto_utils import BotoAWSRequestsAuth

host = 'https://search-lmi-capstone-test-bbewbwsli4uchb7r24zujwrqne.us-east-1.es.amazonaws.com'
# auth = BotoAWSRequestsAuth(aws_host=host,
#                            aws_region='us-east-1',
#                            aws_service='es')
# awsauth = AWS4Auth(YOUR_ACCESS_KEY, YOUR_SECRET_KEY, REGION, 'es')

In [3]:
es = Elasticsearch(host)

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
print(es.info())

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# print(es.info())

{'name': 'dedbbb462bf60acdb6c63621c8bf117c', 'cluster_name': '846033058400:lmi-capstone-test', 'cluster_uuid': 'vhqZptiWQEq0BVgaN6qi8Q', 'version': {'number': '7.8.0', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-10-15T22:09:28.519938Z', 'build_snapshot': False, 'lucene_version': '8.5.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


### Define Rule Splits

In [18]:
rule_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Rule/CMS-2018-0101-0001.txt"

In [49]:
def splitRule_headers(rule_url):

    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    rulechunks = {}
    
    startdict = {'a2':['2. proposals for modified participation options under 5-year agreement periods', '3. creating a basic track with glide path to performance-based risk'], \
     'a3':['3. creating a basic track with glide path to performance-based risk', '4. permitting annual participation elections'], \
    'a4b':['b. proposals for permitting election of differing levels of risk within the basic track\'s glide path', 'c. proposals for permitting annual election of beneficiary assignment methodology'], \
    'a4c':['c. proposals for permitting annual election of beneficiary assignment methodology', '5. determining participation options based on medicare ffs revenue and prior participation '], \
    'a5b':['b. differentiating between low revenue acos and high revenue acos', 'c. determining participation options based on prior participation of aco legal entity and aco participants'], \
    'a5c':['c. determining participation options based on prior participation of aco legal entity and aco participants', 'd. monitoring for financial performance'], \
    'a5d': ['d. monitoring for financial performance', '6. requirements for aco participation in two-sided models'], \
    'a6b': ['b. election of msr/mlr by acos', 'c. aco repayment mechanisms'], \
    'a6c': ['c. aco repayment mechanisms', 'd. advance notice for and payment consequences of termination '], \
    'a6d2': ['(2) proposals for advance notice of voluntary termination', '(3) proposals for payment consequences of termination'], \
    'a6d3': ['(3) proposals for payment consequences of termination', '7. participation options for agreement periods beginning in 2019'], \
    'a7b': ['b. methodology for determining financial and quality performance for the 6-month performance years during 2019', 'c. applicability of program policies to acos participating in a 6-month performance year'], \
    'a7c': ['c. applicability of program policies to acos participating in a 6-month performance year', 'b. fee-for-service benefit enhancements'], \
    'b2a': ['a. shared savings program snf 3-day rule waiver', 'b. billing and payment for telehealth services'], \
    'b2b': ['b. billing and payment for telehealth services', 'c. providing tools to strengthen beneficiary engagement'], \
    'c2': ['2. beneficiary incentives', '3. empowering beneficiary choice'], \
    'c3a': ['3. empowering beneficiary choice', 'b. beneficiary opt-in based assignment methodology'], \
    'c3b': ['b. beneficiary opt-in based assignment methodology', 'd. benchmarking methodology refinements'],  \
    'd2': ['2. risk adjustment methodology for adjusting historical benchmark each performance year', '3. use of regional factors when establishing and resetting acos\' benchmarks'], \
    'd3b': ['b. proposals to apply regional expenditures in determining the benchmark for an aco\'s first agreement period', 'c. proposals for modifying the regional adjustment'], \
    'd3c': ['c. proposals for modifying the regional adjustment', 'd. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark'], \
    'd3d': ['d. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark', '4. technical changes to incorporate references to benchmark rebasing policies'], \
    'd4': ['4. technical changes to incorporate references to benchmark rebasing policies', 'e. updating program policies'], \
    'e2': ['2. revisions to policies on voluntary alignment', '3. revisions to the definition of primary care services used in beneficiary assignment'], \
    'e3': ['3. revisions to the definition of primary care services used in beneficiary assignment', '4. extreme and uncontrollable circumstances policies for the shared savings program'], \
    'e4': ['4. extreme and uncontrollable circumstances policies for the shared savings program', '5. program data and quality measures'], \
    'e5':['5. program data and quality measures', '6. promoting interoperability'], \
    'e6':['6. promoting interoperability', '7. coordination of pharmacy care for aco beneficiaries'], \
    'e7':['7. coordination of pharmacy care for aco beneficiaries', 'f. applicability of proposed policies to track 1+ model acos'], \
    'f2':['2. unavailability of application cycles for entry into the track 1+ model in 2019 and 2020', '3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements'], \
    'f3':['3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements', 'g. summary of proposed timing of applicability']}
    
    for key, value in startdict.items():    
       splitlist = splitlist.split(value[0]) #split on start of desired section
       split_further = splitlist[1].split(value[1]) #split again on start of undesired section
       rulechunks[key] = {"text": split_further[0]} #choose only first half to upload to dict
       splitlist = splitlist[1] #choose second half to prepare for next split
    
    #print(rulechunks.keys())   
    #print(rulechunks["f3"])
    
    #lengths = [len(chunk) for chunk in rulechunks.values()]
    #print(lengths) #characters
    
    return rulechunks

In [21]:
def splitRule_chars(rule_url):
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #chunk by num chars
    chunklist = [(splitlist[i:i+8000]) for i in range(0, len(splitlist), 8000)]
    
    #make dict
    rulechunks = {}
    keys = range(len(chunklist))
    for i in keys:
        rulechunks[i] = {"text": chunklist[i]}

    return rulechunks

In [98]:
def splitRule_headchars(rule_url):
    chunks = splitRule_headers(rule_url) #call first method
    for key, value in chunks.items():
        if len(value["text"]) > 50000: #check to see if chunk is really long
            smalldict = {}
            chunklist = [value["text"][i:i+25000] for i in range(0, len(value["text"]), 25000)] #split chunk
            keys = range(len(chunklist))
            for i in keys:
                smalldict[key+str(i)] = {"text": chunklist[i]} #add to sub-dictionary
            chunks[key] = smalldict #add to big dictionary
        else:
            chunks[key] = {key+str(0): value} #add to big dictionary
    return chunks

### Uploading Rule Splits to ES

In [35]:
def rulesplit_toES(rulechunks, es_index, idnum):
    for key, value in rulechunks.items():
            res = es.index(index=es_index, id=idnum, body={key:value}, doc_type='_doc')
            idnum += 1
            es.indices.refresh(index=es_index)
    print(f"Last id uploaded: {idnum-1}")

In [78]:
def ruletoES_hybrid(rulechunks, es_index, idnum):
    for key, value in rulechunks.items():
        for small_key, small_value in value.items():
            res = es.index(index=es_index, id=idnum, body={small_key:small_value})
            idnum += 1
            es.indices.refresh(index=es_index)
    print(f"Last id uploaded: {idnum-1}")

In [73]:
rulesplit_test = splitRule_headers(rule_url)
test_upload = rulesplit_toES(rulesplit_test, "headers_new", 1)

Last id uploaded: 31


In [85]:
rulesplit_test2 = splitRule_chars(rule_url)
test_upload2 = rulesplit_toES(rulesplit_test2, "char_8000", 1)

Last id uploaded: 111


In [103]:
rulesplit_test3 = splitRule_headchars(rule_url)
test_upload3 = ruletoES_hybrid(rulesplit_test3, "hybrid_new", 1)

Last id uploaded: 90


### Querying ES

In [104]:
#ES Search
def ES_search(index, querydict):
    search = es.search(index=index, doc_type="_doc", body={"query": querydict})
    test_list = []
    if search['hits']['hits'] != []:
        for h in search['hits']['hits']:
            key = list((h['_source'].keys()))
            test_list.append({key[0]:h['_score']})
    return test_list

In [105]:
test_query = {"simple_query_string": {"query": "urban"}}

In [ ]:
headers_test_search = ES_search("headers_new", test_query)
headers_test_search

In [ ]:
char_test_search = ES_search("char_8000", test_query)
char_test_search

In [ ]:
hybrid_test_search = ES_search("hybrid_new", test_query)
hybrid_test_search

### More Utilities:

In [14]:
# Delete an index:
# es.indices.delete(index='index_name', ignore=[400, 404])